# Containers Everywhere, New Problems
- How do we automate container lifecycle?
- How can we easily scale out/in/up/down?
- How can we ensure our containers are re-created if they fail?
- How can we replace containers without downtime (blue/green deploy)?
- How can we control/track where containers get started?
- How can we create cross-node virtual networks?
- How can we ensure only trusted servers run our containers?
- How can we store secrets, keys, passwords and get them to the righ container (and only that container)?

# Swarm Mode: Built-In Orchestration
- Swarm Mode is a clustering solution built inside Docker
- Not related to Swarm `classic` for pre-1.12 versions
- Added in 1.12 via SwarmKit toolkit
- Enhances in 1.13 via Stacks and Secrets
- Not enabled by default, new commands once enabled
    - docker swarm
    - docker node
    - docker service
    - docker stack
    - docker secret

In [1]:
# Single Node - Swarm

In [2]:
!docker info

Client:
 Debug Mode: false
 Plugins:
  scan: Docker Scan (Docker Inc., v0.3.4)

Server:
 Containers: 16
  Running: 0
  Paused: 0
  Stopped: 16
 Images: 113
 Server Version: 19.03.13
 Storage Driver: overlay2
  Backing Filesystem: extfs
  Supports d_type: true
  Native Overlay Diff: true
 Logging Driver: json-file
 Cgroup Driver: cgroupfs
 Plugins:
  Volume: local
  Network: bridge host ipvlan macvlan null overlay
  Log: awslogs fluentd gcplogs gelf journald json-file local logentries splunk syslog
 Swarm: inactive
 Runtimes: runc
 Default Runtime: runc
 Init Binary: docker-init
 containerd version: 8fba4e9a7d01810a393d5d25a3621dc101981175
 runc version: dc9208a3303feef5b3839f4323d9beb36df0a9dd
 init version: fec3683
 Security Options:
  seccomp
   Profile: default
 Kernel Version: 5.4.39-linuxkit
 Operating System: Docker Desktop
 OSType: linux
 Architecture: x86_64
 CPUs: 4
 Total Memory: 1.941GiB
 Name: docker-desktop
 ID: 3KKU:3QWO:3MBQ:G2Y2:HANM:CRAL:HQRB:ZNBY:O7JZ:NMNT:I6KB:KVGT
 

In [3]:
!docker swarm init

Swarm initialized: current node (yso27q4msispya67ime9r1e8w) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-2o76mshrg52pug8bn24gh6boycw4yoi24h599g5amr5t8v3g9w-9gb3btsqz387h2l8m8v6k2vmh 192.168.65.3:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.



- Lots of PKI(public key initalization) and security automation
    - Root signing certificate created for our Swarm
    - Certificate is issued for first Manager node
    - Join tokens are created
- Raft database created to store root CA, configs and secrets
    - Encrypted to default disk(1.13+)
    - No need for another key/value system to hold orchestration/secrets
    - Replicas logs amongst Managers via mutual TLS in `control plane`

In [4]:
!docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS      ENGINE VERSION
yso27q4msispya67ime9r1e8w *   docker-desktop      Ready               Active              Leader              19.03.13


In [6]:
!docker node --help


Usage:	docker node COMMAND

Manage Swarm nodes

Commands:
  demote      Demote one or more nodes from manager in the swarm
  inspect     Display detailed information on one or more nodes
  ls          List nodes in the swarm
  promote     Promote one or more nodes to manager in the swarm
  ps          List tasks running on one or more nodes, defaults to current node
  rm          Remove one or more nodes from the swarm
  update      Update a node

Run 'docker node COMMAND --help' for more information on a command.


In [7]:
!docker swarm --help


Usage:	docker swarm COMMAND

Manage Swarm

Commands:
  ca          Display and rotate the root CA
  init        Initialize a swarm
  join        Join a swarm as a node and/or manager
  join-token  Manage join tokens
  leave       Leave the swarm
  unlock      Unlock swarm
  unlock-key  Manage the unlock key
  update      Update the swarm

Run 'docker swarm COMMAND --help' for more information on a command.


In [8]:
!docker service --help


Usage:	docker service COMMAND

Manage services

Commands:
  create      Create a new service
  inspect     Display detailed information on one or more services
  logs        Fetch the logs of a service or task
  ls          List services
  ps          List the tasks of one or more services
  rm          Remove one or more services
  rollback    Revert changes to a service's configuration
  scale       Scale one or multiple replicated services
  update      Update a service

Run 'docker service COMMAND --help' for more information on a command.


In [9]:
!docker service create alpine ping 8.8.8.8

ibtxoaovbvuvd7zgbznnm15v7

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
fy: Service converged to verify that tasks are stable... 

In [10]:
!docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
ibtxoaovbvuv        busy_lewin          replicated          1/1                 alpine:latest       


In [11]:
!docker service ps ibtxoaovbvuv

ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE                ERROR               PORTS
ji686uak3s5z        busy_lewin.1        alpine:latest       docker-desktop      Running             Running about a minute ago                       


In [12]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED              STATUS              PORTS               NAMES
d1cf3816db8d        alpine:latest       "ping 8.8.8.8"      About a minute ago   Up About a minute                       busy_lewin.1.ji686uak3s5zip69zihxgjjym


In [13]:
!docker service update ibtxoaovbvuv --replicas 3

ibtxoaovbvuv

all progress: 0 out of 3 tasks 
   K
   K
all progress: 3 out of 3 tasks 
fy: Service converged to verify that tasks are stable... 

In [14]:
!docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
ibtxoaovbvuv        busy_lewin          replicated          3/3                 alpine:latest       


In [16]:
!docker service ps ibtxoaovbvuv 

ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE            ERROR               PORTS
ji686uak3s5z        busy_lewin.1        alpine:latest       docker-desktop      Running             Running 2 minutes ago                        
pm1xhk6hlquq        busy_lewin.2        alpine:latest       docker-desktop      Running             Running 41 seconds ago                       
pr4fk3wdzazg        busy_lewin.3        alpine:latest       docker-desktop      Running             Running 41 seconds ago                       


In [17]:
!docker update --help


Usage:	docker update [OPTIONS] CONTAINER [CONTAINER...]

Update configuration of one or more containers

Options:
      --blkio-weight uint16        Block IO (relative weight), between 10
                                   and 1000, or 0 to disable (default 0)
      --cpu-period int             Limit CPU CFS (Completely Fair
                                   Scheduler) period
      --cpu-quota int              Limit CPU CFS (Completely Fair
                                   Scheduler) quota
      --cpu-rt-period int          Limit the CPU real-time period in
                                   microseconds
      --cpu-rt-runtime int         Limit the CPU real-time runtime in
                                   microseconds
  -c, --cpu-shares int             CPU shares (relative weight)
      --cpus decimal               Number of CPUs
      --cpuset-cpus string         CPUs in which to allow execution (0-3, 0,1)
      --cpuset-mems string         MEMs in which to allow execution (0-3,

In [18]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
8ff45f581628        alpine:latest       "ping 8.8.8.8"      2 minutes ago       Up 2 minutes                            busy_lewin.3.pr4fk3wdzazg8ayo4cb5m2p3x
3bfcd96e36d8        alpine:latest       "ping 8.8.8.8"      2 minutes ago       Up 2 minutes                            busy_lewin.2.pm1xhk6hlquqyhzaiyps4oypl
d1cf3816db8d        alpine:latest       "ping 8.8.8.8"      4 minutes ago       Up 4 minutes                            busy_lewin.1.ji686uak3s5zip69zihxgjjym


In [22]:
!docker container rm -f busy_lewin.3.pr4fk3wdzazg8ayo4cb5m2p3x

Error: No such container: busy_lewin.3.pr4fk3wdzazg8ayo4cb5m2p3x


In [23]:
!docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
ibtxoaovbvuv        busy_lewin          replicated          3/3                 alpine:latest       


In [24]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
18366390c4af        alpine:latest       "ping 8.8.8.8"      35 seconds ago      Up 29 seconds                           busy_lewin.3.vcyj3i5wo9myyqev3q0yx9f91
3bfcd96e36d8        alpine:latest       "ping 8.8.8.8"      4 minutes ago       Up 4 minutes                            busy_lewin.2.pm1xhk6hlquqyhzaiyps4oypl
d1cf3816db8d        alpine:latest       "ping 8.8.8.8"      6 minutes ago       Up 6 minutes                            busy_lewin.1.ji686uak3s5zip69zihxgjjym


In [25]:
!docker service ps busy_lewin

ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE                ERROR                         PORTS
ji686uak3s5z        busy_lewin.1        alpine:latest       docker-desktop      Running             Running 6 minutes ago                                      
pm1xhk6hlquq        busy_lewin.2        alpine:latest       docker-desktop      Running             Running 4 minutes ago                                      
vcyj3i5wo9my        busy_lewin.3        alpine:latest       docker-desktop      Running             Running about a minute ago                                 
pr4fk3wdzazg         \_ busy_lewin.3    alpine:latest       docker-desktop      Shutdown            Failed about a minute ago    "task: non-zero exit (137)"   


In [26]:
!docker  service rm busy_lewin

busy_lewin


In [28]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [29]:
!docker version

Client: Docker Engine - Community
 Cloud integration: 1.0.2
 Version:           19.03.13
 API version:       1.40
 Go version:        go1.13.15
 Git commit:        4484c46d9d
 Built:             Wed Sep 16 16:58:31 2020
 OS/Arch:           darwin/amd64
 Experimental:      false

Server: Docker Engine - Community
 Engine:
  Version:          19.03.13
  API version:      1.40 (minimum version 1.12)
  Go version:       go1.13.15
  Git commit:       4484c46d9d
  Built:            Wed Sep 16 17:07:04 2020
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          v1.3.7
  GitCommit:        8fba4e9a7d01810a393d5d25a3621dc101981175
 runc:
  Version:          1.0.0-rc10
  GitCommit:        dc9208a3303feef5b3839f4323d9beb36df0a9dd
 docker-init:
  Version:          0.18.0
  GitCommit:        fec3683


In [30]:
!docker-machine version

zsh:1: command not found: docker-machine
